## Prepare the data for the model used in the paper ##

Read each file and segment them into one hour windows. Each file will contain 28,800 samples per hour at an 8 Hz sampling rate.

In [ ]:
import os
from obspy import read
import glob
import numpy as np
from obspy import UTCDateTime

# Define the processed and output folder paths
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define the target segment duration (1 hour) in seconds
segment_duration = 3600  # seconds

# Process each processed file for segmentation
for file_path in glob.glob(f"{processed_folder}/*.mseed"):
    # Read the processed file
    st = read(file_path)
    
    # Normalize the traces in the stream object - done in data_cleaning
    #st.normalize()
    
    # Split each trace in the Stream object into one-hour segments
    for tr in st:
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time
        
        # Loop over each one-hour segment
        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            # Format the filename for the segment
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.mseed"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)
            
            # Save the one-hour segment as a new file
            try:
                segment.write(segment_filepath, format="MSEED")
                print(f"Saved segment file: {segment_filename}")
            except Exception as e:
                print(f"Error writing segment {segment_filename}: {e}")

            # Move to the next hour
            segment_start = segment_end

            # # Save the one-hour segment as a new file
            # segment.write(segment_filepath, format="MSEED")
            # print(f'Saved segment file: {segment_filepath}')

            # # Move to the next hour
            # segment_start = segment_end

In [ ]:
# check all files in the segmented folder for nan data

for file_path in glob.glob(f"{segmented_output_folder}/*.mseed"):
    st = read(file_path)
    for tr in st:
        if np.isnan(tr.data).any():
            print(f"File {file_path} contains NaN data")
        # else:
        #     print(f"File {file_path} is clean")
print(f'done')

Apply the Fourier Transform with the `specified window_length` and `hop_length parameters` to produce a spectrogram of size (96, 128) for each one-hour segment. The data is saved in `data\segmented` from the cell above.

In [ ]:
import os
import glob
import numpy as np
from obspy import read
import librosa
from scipy.signal import resample
import time
from IPython.display import display, clear_output
import torch

# Define directories
input_folder = os.getcwd() + '/data/raw'
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define constants
target_sampling_rate = 8  # Hz
segment_duration = 3600  # 1 hour in seconds
window_length = 256
hop_length = 224
target_shape = (96, 128)

# Initialize an empty list to store spectrograms
spectrogram_list = []

# Step 1: Process each file in the input folder
for file_path in glob.glob(f"{input_folder}/*.mseed"):
    # Read the file
    st = read(file_path)
    
    # Process each trace
    for tr in st:
        # Resample the data to 8 Hz (assuming original sampling rate was 100 Hz)
        # this is done in data cleaning
        #tr.resample(target_sampling_rate)
        
        # Split into one-hour segments (28,800 samples each)
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time

        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            """
            # Compute STFT for the segment
            signal = segment.data.astype(np.float32)
            stft_result = librosa.stft(signal, n_fft=window_length, hop_length=hop_length, win_length=window_length)
            
            # use obspy stft
            #stft_result = segment.spectrogram(wlen=window_length, per_lap=0.9, log=False)
            spectrogram = np.abs(stft_result) ** 2

            # Resize spectrogram to target shape
            spectrogram_resized = librosa.util.fix_length(spectrogram, size=target_shape[1], axis=1)[:target_shape[0], :]
            
            # Apply log transformation first
            spectrogram_resized = np.maximum(spectrogram_resized, 1e-12)
            spectrogram_log = np.log1p(spectrogram_resized)

            # Normalize the log-transformed data between -1 and 1
            spectrogram_log = 2 * (spectrogram_log - np.min(spectrogram_log)) / np.ptp(spectrogram_log) - 1
            
            """
            ## NEW ##
            # Convert to PyTorch tensor
            segment_tensor = torch.tensor(segment, dtype=torch.float32)
            
            # Ensure correct dimensions (STFT expects 1D tensor or batch x time)
            if len(segment_tensor.shape) == 1:
                segment_tensor = segment_tensor.unsqueeze(0)
                
            # Compute the STFT
            stft_result = torch.stft(
                segment_tensor,
                n_fft=window_length,
                hop_length=hop_length,
                win_length=window_length,
                return_complex=True,
            )
                
            # Compute the spectrogram (magnitude squared)
            spectrogram = torch.abs(stft_result) ** 2

            # Convert to NumPy for further processing
            spectrogram = spectrogram.numpy()

            # Resize to a consistent shape if needed (example using interpolation)
            target_shape = (96, 128)  # Desired shape
            spectrogram_resized = np.resize(spectrogram, target_shape)

            # Log-transform the spectrogram
            spectrogram_log = np.log1p(spectrogram_resized)

            # Normalize between -1 and 1
            spectrogram_log = 2 * (spectrogram_log - np.min(spectrogram_log)) / np.ptp(spectrogram_log) - 1
                    ## END NEW ##

            # Save the spectrogram (you could save the result as needed)
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.npy"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)

            np.save(segment_filepath, spectrogram_log)

            # Append the spectrogram to the list
            spectrogram_list.append(spectrogram_log)
            
            # check the file for data outside the bounds of -1 and 1
            if np.any(spectrogram_log > 1) or np.any(spectrogram_log < -1):
                print(f"File {segment_filename} contains data outside the bounds of -1 and 1")

            # Move to the next hour
            segment_start = segment_end
            
            # display file being processed
            # time.sleep(0.1)
            # clear_output(wait=True)
            print(f'Processing file: {segment_filename}')
            
            #check file for nan data
            if np.isnan(spectrogram_log).any():
                print(f"File {segment_filename} contains NaN data")             

# Stack all spectrograms into a single numpy array and add batch and channel dimensions
all_spectrograms = np.stack(spectrogram_list)
all_spectrograms = all_spectrograms[:, np.newaxis, :, :]  # Shape: (batch_size, 1, 96, 128)
print(f"Shape of combined spectrogram array: {all_spectrograms.shape}")

# remove the single feature dimension (1)
all_spectrograms = np.squeeze(all_spectrograms)

# NOTE: FILE NAME IS HARD CODED HERE
# Save the combined spectrogram array to Input.npy
final_output_folder = os.getcwd()
input_filepath = os.path.join(final_output_folder, "Input.npy")
#input_filepath = os.path.join(final_output_folder + 'NUPH_analysis', "Input_nuph.npy")
np.save(input_filepath, all_spectrograms)

print(f"Saved combined spectrogram file: {input_filepath.split('/')[-1]}")


In [ ]:
# import matplotlib.pyplot as plt

# # Ensure there are spectrograms to plot
# if all_spectrograms.size > 0:
#     length = len(all_spectrograms)
#     print(length)
#     plt.figure(figsize=(10, 6))
#     for i in range(length):  # Loop through all spectrograms
#         plt.plot(all_spectrograms[i].flatten())
#     plt.title(f'Spectrograms as Line Plot')
#     plt.xlabel('Frequency Bins')
#     plt.ylabel('Log Power')
#     plt.show()
# else:
#     print("No spectrograms found in all_spectrograms.")

In [ ]:
#display information on Input.npy
print(f"Shape of combined spectrogram array with batch and channel dimensions: {all_spectrograms.shape}")
print(f"Number of files processed: {all_spectrograms.shape[0]}")

# load Input.npy and display information for confirmation
input_data_file_t = np.load(input_filepath)
print(f"Shape of combined spectrogram array Input.npy: {input_data_file_t.shape}")
print(f"Number of files processed: {input_data_file_t.shape[0]}")

In [ ]:
# check the data in all_spectograms for any nans
nan_check = np.isnan(all_spectrograms)
nan_count = np.sum(nan_check)
print(f"Number of NaNs in all_spectrograms: {nan_count}")

# how much data is NOT nan?
nan_count = np.sum(~nan_check)
print(f"Number of non-NaNs in all_spectrograms: {nan_count}")

In [ ]:
# display first five rows in the all_spectograms
print(all_spectrograms[:5])
